In [1]:
import json
import yaml
import uuid
import os
from pathlib import Path

In [2]:
import pandas as pd

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import wombat.pecgs as pecgs

###### make cwl from bam wxs and rnaseq tumor fastq

In [11]:
cwl_dir = '/home/estorrs/pecgs-pipeline/cwl/pecgs_workflows'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [9]:
template = {
    'sample': 'a_string',
    'cpu': 'a_int',
    'tumor_wxs_bam': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'normal_wxs_bam': {
        'class': 'File', 
        'path': 'a/file/path'
    },
    'tumor_rna_fq_1': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'tumor_rna_fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    
    'reference': {
        'class': 'File',
        'path': 'path/to/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.dict'}
        ]
    },
    
    # fusion
    'genome_lib_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'genome_db': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'bwts': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'integrate_executable': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_fasta': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_annotations': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'filter_database': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'fusion_annotator_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    
    # CNV
    'reference_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'target_interval_list': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'common_biallelic': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'protein_coding_gene': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'pool_of_normals': {
        'class': 'File',
        'path': 'a/file/path'
    },
    
    # msisensor
    'microsatellite': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'minimal_homopolymer_size': 'a_int',
    'minimal_microsatellite_size': 'a_int',
    
    # TinDaisy
    'rescue_clinvar': False,
    'rescue_cosmic': False,
    'vep_cache_version': 'a_string',
    'vep_cache_gz': {'class': 'File', 'path': 'a/file/path'},
    'clinvar_annotation': {'class': 'File', 'path': 'a/file/path'},
    'classification_config': {'class': 'File', 'path': 'a/file/path'},
    'af_config': {'class': 'File', 'path': 'a/file/path'},
    'call_regions': {'class': 'File', 'path': 'a/file/path'},
    'canonical_BED': {'class': 'File', 'path': 'a/file/path'},
    'normal_barcode': 'a_string',
    'tumor_barcode': 'a_string',
    'tindaisy_chrlist': {'class': 'File', 'path': 'a/file/path'},
    'strelka_config': {'class': 'File', 'path': 'a/file/path'},
    'centromere_bed': {'class': 'File', 'path': 'a/file/path'},
    'assembly': 'a_string',
    'varscan_config': {'class': 'File', 'path': 'a/file/path'},
    'pindel_config': {'class': 'File', 'path': 'a/file/path'},
    
    # TinJasmine
    'sample_barcode': 'a_string',
    'centromere': {'class': 'File', 'path': 'a/file/path'},
    'tinjasmine_chrlist': {'class': 'File', 'path': 'a/file/path'},
    'Canonical_BED': {'class': 'File', 'path': 'a/file/path'},
    'ROI_BED': {'class': 'File', 'path': 'a/file/path'},
    'varscan_filter_config': {'class': 'File', 'path': 'a/file/path'},
    'pindel_filter_config': {'class': 'File', 'path': 'a/file/path'},
    'pindel_config_template': {'class': 'File', 'path': 'a/file/path'},
    'gatk_filter_config': {'class': 'File', 'path': 'a/file/path'}, 

}

In [12]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.pecgs_TN_wxs_bam_T_rna_fq.yaml'), 'w'))

In [13]:
cwl = {
    'cwlVersion': 'v1.0',
    'class': 'Workflow',
    'id': 'pecgs_TN_wxs_bam_T_rna_fq',
    'label': 'pecgs_TN_wxs_bam_T_rna_fq',
    'inputs': [
        {
            'id': 'sample',
            'type': 'string',
        },
        {
            'id': 'cpu',
            'type': 'int?',
            'default': 40
        },
        {
            'id': 'tumor_wxs_bam',
            'type': 'File'
        },
        {
            'id': 'normal_wxs_bam',
            'type': 'File'
        },
        {
            'id': 'tumor_rna_fq_1',
            'type': 'File'
        },
        {
            'id': 'tumor_rna_fq_2',
            'type': 'File'
        },
        
        {
            'id': 'reference',
            'type': 'File',
            'secondaryFiles': [
               '.amb',
               '.ann',
               '.bwt',
               '.fai',
               '.pac',
               '.sa',
               '^.dict'
           ]
        },
        
        # fusion
        {
            'id': 'genome_lib_dir',
            'type': 'Directory',
        },
        {
            'id': 'genome_db',
            'type': 'Directory',
        },
        {
            'id': 'bwts',
            'type': 'Directory',
        },
        {
            'id': 'integrate_executable',
            'type': 'File',
        },
        {
            'id': 'integrate_fasta',
            'type': 'File',
        },
        {
            'id': 'integrate_annotations',
            'type': 'File',
        },
        {
            'id': 'filter_database',
            'type': 'Directory',
        },
        {
            'id': 'fusion_annotator_dir',
            'type': 'Directory',
        },
        
        # CNV
        {
            'id': 'reference_dir',
            'type': 'Directory',
        },
        {
            'id': 'target_interval_list',
            'type': 'File',
        },
        {
            'id': 'common_biallelic',
            'type': 'File',
        },
        {
            'id': 'protein_coding_gene',
            'type': 'File',
        },
        {
            'id': 'pool_of_normals',
            'type': 'File',
        },
        
        # msisensor
        {
            'id': 'microsatellite',
            'type': 'File',
        },
        {
            'id': 'minimal_homopolymer_size',
            'type': 'int?',
            'default': 1
        },
        {
            'id': 'minimal_microsatellite_size',
            'type': 'int?',
            'default': 1
        },
        
        # TinDaisy
        {
            'id': 'rescue_clinvar',
            'type': 'boolean?',
            'default': False,
        },
        {
            'id': 'rescue_cosmic',
            'type': 'boolean?',
            'default': False,
        },
        {
            'id': 'vep_cache_version',
            'type': 'string'
        },
        {
            'id': 'vep_cache_gz',
            'type': 'File'
        },
        {
            'id': 'clinvar_annotation',
            'type': 'File'
        },
        {
            'id': 'classification_config',
            'type': 'File'
        },
        {
            'id': 'af_config',
            'type': 'File'
        },
        {
            'id': 'call_regions',
            'type': 'File'
        },
        {
            'id': 'canonical_BED',
            'type': 'File'
        },
        {
            'id': 'normal_barcode',
            'type': 'string?',
            'default': '$(inputs.sample).N',
        },
        {
            'id': 'tumor_barcode',
            'type': 'string?',
            'default': '$(inputs.sample).T',
        },
        {
            'id': 'tindaisy_chrlist',
            'type': 'File'
        },
        {
            'id': 'strelka_config',
            'type': 'File'
        },
        {
            'id': 'centromere_bed',
            'type': 'File'
        },
        {
            'id': 'assembly',
            'type': 'string'
        },
        {
            'id': 'varscan_config',
            'type': 'File'
        },
        {
            'id': 'pindel_config',
            'type': 'File'
        },
        
        # TinJasmine
        {
            'id': 'sample_barcode',
            'type': 'string?',
            'default': '$(inputs.sample)'
        },
        {
            'id': 'centromere',
            'type': 'File'
        },
        {
            'id': 'tinjasmine_chrlist',
            'type': 'File'
        },
        {
            'id': 'Canonical_BED',
            'type': 'File'
        },
        {
            'id': 'ROI_BED',
            'type': 'File'
        },
        {
            'id': 'varscan_filter_config',
            'type': 'File'
        },
        {
            'id': 'pindel_filter_config',
            'type': 'File'
        },
        {
            'id': 'pindel_config_template',
            'type': 'File'
        },
        {
            'id': 'gatk_filter_config',
            'type': 'File'
        }
    ],
    'outputs': [
        {
            'id': 'filtered_fusions',
            'type': 'File',
            'outputSource': 'run_fusion/filtered_fusions'
        },
        {
            'id': 'total_fusions',
            'type': 'File',
            'outputSource': 'run_fusion/total_fusions'
        },
        {
            'id': 'gene_level_cnv',
            'type': 'File',
            'outputSource': 'run_cnv/gene_level_cnv'
        },
        {
            'id': 'msisensor_output_summary',
            'type': 'File',
            'outputSource': 'run_msisensor/output_summary'
        },
        {
            'id': 'msisensor_output_dis',
            'type': 'File',
            'outputSource': 'run_msisensor/output_dis'
        },
        {
            'id': 'msisensor_output_germline',
            'type': 'File',
            'outputSource': 'run_msisensor/output_germline'
        },
        {
            'id': 'msisensor_output_somatic',
            'type': 'File',
            'outputSource': 'run_msisensor/output_somatic'
        },
        {
            'id': 'tindaisy_output_maf_clean',
            'type': 'File',
            'outputSource': 'run_tindaisy/output_maf_clean'
        },
        {
            'id': 'tindaisy_output_vcf_clean',
            'type': 'File',
            'outputSource': 'run_tindaisy/output_vcf_clean'
        },
        {
            'id': 'tindaisy_output_vcf_all',
            'type': 'File',
            'outputSource': 'run_tindaisy/output_vcf_all'
        },
        {
            'id': 'tinjasmine_output_maf_clean',
            'type': 'File',
            'outputSource': 'run_tinjasmine/clean_MAF'
        },
        {
            'id': 'tinjasmine_output_vcf_clean',
            'type': 'File',
            'outputSource': 'run_tinjasmine/clean_VCF'
        },
        {
            'id': 'tinjasmine_output_vcf_all',
            'type': 'File',
            'outputSource': 'run_tinjasmine/allCall_VCF'
        },
    ],
    'steps': [
        {
            'id': 'run_fusion',
            'label': 'run_fusion',
            'run': '../../submodules/pecgs-fusion/cwl/fusion.cwl',
            'in': [
                {'id': 'sample', 'source': 'sample'},
                {'id': 'cpu', 'source': 'cpu'},
                {'id': 'fq_1', 'source': 'tumor_rna_fq_1'},
                {'id': 'fq_2', 'source': 'tumor_rna_fq_2'},
                {'id': 'filter_database', 'source': 'filter_database'},
                {'id': 'bwts', 'source': 'bwts'},
                {'id': 'fusion_annotator_dir', 'source': 'fusion_annotator_dir'},
                {'id': 'genome_db', 'source': 'genome_db'},
                {'id': 'genome_lib_dir', 'source': 'genome_lib_dir'},
                {'id': 'integrate_annotations', 'source': 'integrate_annotations'},
                {'id': 'integrate_executable', 'source': 'integrate_executable'},
                {'id': 'integrate_fasta', 'source': 'integrate_fasta'}
            ],
            'out': [
                {'id': 'filtered_fusions'},
                {'id': 'total_fusions'}
            ]
        },
        {
            'id': 'run_cnv',
            'label': 'run_cnv',
            'run': '../../submodules/pecgs-cnv/cwl/cnv_workflow.cwl',
            'in': [
                {'id': 'sample', 'source': 'sample'},
                {'id': 'cpu', 'source': 'cpu'},
                {'id': 'tumor_bam', 'source': 'tumor_wxs_bam'},
                {'id': 'normal_bam', 'source': 'normal_wxs_bam'},
                {'id': 'reference_dir', 'source': 'reference_dir'},
                {'id': 'target_interval_list', 'source': 'target_interval_list'},
                {'id': 'common_biallelic', 'source': 'common_biallelic'},
                {'id': 'protein_coding_gene', 'source': 'protein_coding_gene'},
                {'id': 'pool_of_normals', 'source': 'pool_of_normals'},
            ],
            'out': [
                {'id': 'gene_level_cnv'}
            ]
        },
        {
            'id': 'run_msisensor',
            'label': 'run_msisensor',
            'run': '../msisensor/msisensor_workflow.cwl',
            'in': [
                {'id': 'threads', 'source': 'cpu'},
                {'id': 'tumor_bam', 'source': 'tumor_wxs_bam'},
                {'id': 'normal_bam', 'source': 'normal_wxs_bam'},
                {'id': 'microsatellite', 'source': 'microsatellite'},
                {'id': 'minimal_homopolymer_size', 'source': 'minimal_homopolymer_size'},
                {'id': 'minimal_microsatellite_size', 'source': 'minimal_microsatellite_size'},
            ],
            'out': [
                {'id': 'output_summary'},
                {'id': 'output_dis'},
                {'id': 'output_germline'},
                {'id': 'output_somatic'},
            ]
        },
        {
            'id': 'run_tindaisy',
            'label': 'run_tindaisy',
            'run': '../../submodules/TinDaisy/cwl/workflows/tindaisy2.cwl',
            'in': [
                {'id': 'tumor_bam', 'source': 'tumor_wxs_bam'},
                {'id': 'normal_bam', 'source': 'tumor_normal_bam'},
                {'id': 'reference_fasta', 'source': 'reference'},
                {'id': 'pindel_config', 'source': 'pindel_config'},
                {'id': 'varscan_config', 'source': 'varscan_config'},
                {'id': 'assembly', 'source': 'assembly'},
                {'id': 'centromere_bed', 'source': 'centromere_bed'},
                {'id': 'strelka_config', 'source': 'strelka_config'},
                {'id': 'chrlist', 'source': 'tindaisy_chrlist'},
                {'id': 'tumor_barcode', 'source': 'tumor_barcode'},
                {'id': 'normal_barcode', 'source': 'normal_barcode'},
                {'id': 'canonical_BED', 'source': 'canonical_BED'},
                {'id': 'call_regions', 'source': 'call_regions'},
                {'id': 'af_config', 'source': 'af_config'},
                {'id': 'classification_config', 'source': 'classification_config'},
                {'id': 'clinvar_annotation', 'source': 'clinvar_annotation'},
                {'id': 'vep_cache_gz', 'source': 'vep_cache_gz'},
                {'id': 'vep_cache_version', 'source': 'vep_cache_version'},
                {'id': 'rescue_cosmic', 'source': 'rescue_cosmic'},
                {'id': 'rescue_clinvar', 'source': 'rescue_clinvar'},
            ],
            'out': [
                {'id': 'output_maf_clean'},
                {'id': 'output_vcf_clean'},
                {'id': 'output_vcf_all'},
            ]
        },
        {
            'id': 'run_tinjasmine',
            'label': 'run_tinjasmine',
            'run': '../../submodules/TinJasmine/cwl/TinJasmine.cwl',
            'in': [
                {'id': 'sample_barcode', 'source': 'normal_barcode'},
                {'id': 'bam', 'source': 'normal_wxs_bam'},
                {'id': 'reference', 'source': 'reference'},
                {'id': 'gatk_filter_config', 'source': 'gatk_filter_config'},
                {'id': 'pindel_config_template', 'source': 'pindel_config_template'},
                {'id': 'pindel_filter_config', 'source': 'pindel_filter_config'},
                {'id': 'varscan_filter_config', 'source': 'varscan_filter_config'},
                {'id': 'ROI_BED', 'source': 'ROI_BED'},
                {'id': 'vep_cache_gz', 'source': 'vep_cache_gz'},
                {'id': 'vep_cache_version', 'source': 'vep_cache_version'},
                {'id': 'assembly', 'source': 'assembly'},
                {'id': 'Canonical_BED', 'source': 'Canonical_BED'},
                {'id': 'chrlist', 'source': 'tinjasmine_chrlist'},
                {'id': 'centromere', 'source': 'centromere'},
            ],
            'out': [
                {'id': 'clean_VCF'},
                {'id': 'allCall_VCF'},
                {'id': 'clean_MAF'},
            ]
        },
        
    ],
   'requirements': [
   ]
}

In [14]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'pecgs_TN_wxs_bam_T_rna_fq.cwl'), 'w'))

###### test run

In [7]:
run_uuid = str(uuid.uuid4())
sample_id = 'MMRF_1250'
data = [
    [f'{sample_id}_{run_uuid}', '/storage1/fs1/dinglab/Active/Projects/multiple_myeloma/MMRF/RNASeq_tumor_FASTQ/SRR1567020_1.fastq.gz', str(uuid.uuid4()), '/storage1/fs1/dinglab/Active/Projects/multiple_myeloma/MMRF/RNASeq_tumor_FASTQ/SRR1567020_2.fastq.gz', str(uuid.uuid4()), '/storage1/fs1/dinglab/Active/Projects/cliu/MMRF/bulk_dna_alignment/20220130_WXS/outputs/job_40/mark_dup_bam/SRR1566924.WXS.N.bam', str(uuid.uuid4()), '/storage1/fs1/dinglab/Active/Projects/cliu/MMRF/bulk_dna_alignment/20220130_WXS/outputs/job_4/mark_dup_bam/SRR1566923.WXS.T.bam', str(uuid.uuid4()), sample_id, run_uuid]
]
run_list = pd.DataFrame(data=data,
                       columns=['run_id', 'rna-seq_tumor_R1.filepath', 'rna-seq_tumor_R1.uuid', 'rna-seq_tumor_R2.filepath', 'rna-seq_tumor_R2.uuid', 'wxs_normal_bam.filepath', 'wxs_normal_bam.uuid', 'wxs_tumor_bam.filepath', 'wxs_tumor_bam.uuid', 'sample_id', 'run_uuid'])
run_list = run_list.set_index('run_id')
run_list

,rna-seq_tumor_R1.filepath,rna-seq_tumor_R1.uuid,rna-seq_tumor_R2.filepath,rna-seq_tumor_R2.uuid,wxs_normal_bam.filepath,wxs_normal_bam.uuid,wxs_tumor_bam.filepath,wxs_tumor_bam.uuid,sample_id,run_uuid
run_id,,,,,,,,,,
MMRF_1250_74405909-4c72-4cca-8991-34d0af397ac1,/storage1/fs1/dinglab/Active/Projects/multiple...,77a6ca24-e9b7-4c2b-b7f3-2ac9fd17f0b9,/storage1/fs1/dinglab/Active/Projects/multiple...,8ee74ac6-a768-4c89-a1ac-0f94b0a337e9,/storage1/fs1/dinglab/Active/Projects/cliu/MMR...,80f84b14-e98c-433b-bf64-7db51d4b21a5,/storage1/fs1/dinglab/Active/Projects/cliu/MMR...,e7395d87-2524-4712-90c4-d555d693f2b8,MMRF_1250,74405909-4c72-4cca-8991-34d0af397ac1


In [17]:
!mkdir -p /home/estorrs/pecgs-pipeline/examples/MMRF_1250

In [18]:
run_list.to_csv('/home/estorrs/pecgs-pipeline/examples/MMRF_1250/runlist.txt', sep='\t')